## Analysis

In [1]:
from time import time
import math, os
import numpy as np
from sklearn import metrics
import h5py
import scanpy as sc
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import sys
sys.path.append('./Baseline_Code')
sys.path.append('./EnhancedscDeepCluster')

In [2]:
# scDeepCluster
from scDeepCluster import scDeepCluster
from single_cell_tools import *
from preprocess import read_dataset, normalize

In [3]:
# ModifiedscDeepCluster
import model
import model_single_cell_tools
import model_preprocess

In [4]:
# for repeatability
torch.manual_seed(42)

Setup parameters.

In [5]:
'''
Parameter setting
'''

class Args(object):
    def __init__(self):
        self.n_clusters = 8
        self.knn = 20
        self.resolution = 0.8
        self.select_genes = 0
        self.batch_size = 256
        self.data_file = '10X_PBMC.h5'
        self.maxiter = 2000
        self.pretrain_epochs = 300
        self.gamma = 1.
        self.sigma = 2.5
        self.update_interval = 1
        self.tol = 0.001
        self.use_attention = True
        self.use_layernorm = False
        self.use_batchnorm = False
        # self.ae_weights = None
        self.ae_weights = '10xPBMC_with_attn_weights.pth.tar'
        self.save_dir = 'results/EnhancedscDeepCluster/'
        self.ae_weight_file = '10xPBMC_with_attn_weights.pth.tar'
        self.final_latent_file = '10xPBMC_final_latent_file.txt'
        self.predict_label_file = '10xPBMC_pred_labels.txt'
        self.device = 'cuda'

args = Args()

Normalizating and preprocessing count data.

In [6]:
data_mat = h5py.File(args.data_file)
x = np.array(data_mat['X'])
# y is the ground truth labels for evaluating clustering performance
# If not existing, we skip calculating the clustering performance metrics (e.g. NMI ARI)
if 'Y' in data_mat:
    y = np.array(data_mat['Y'])
else:
    y = None
data_mat.close()

if args.select_genes > 0:
    importantGenes = model_single_cell_tools.geneSelection(x, n=args.select_genes, plot=False)
    x = x[:, importantGenes]

# preprocessing scRNA-seq read counts matrix
adata = sc.AnnData(x)
if y is not None:
    adata.obs['Group'] = y

adata = model_preprocess.read_dataset(adata,
                 transpose=False,
                 test_split=False,
                 copy=True)

adata = model_preprocess.normalize(adata,
                  size_factors=True,
                  normalize_input=True,
                  logtrans_input=True)

input_size = adata.n_vars

print(args)

print(adata.X.shape)
if y is not None:
    print(y.shape)

### Autoencoder: Successfully preprocessed 16653 genes and 4271 cells.
(4271, 16653)
(4271,)


Build scDeepCluster model

In [7]:
model = model.EnhancedscDeepCluster(input_dim=adata.n_vars, z_dim=32, use_attention=args.use_attention, use_layernorm=args.use_layernorm, use_batchnorm=args.use_batchnorm,
            encodeLayer=[256, 64], decodeLayer=[64, 256], sigma=args.sigma, gamma=args.gamma, device=args.device)

print(str(model))

C:\Users\bhari\Downloads\EnhancedscDeepCluster\project\lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


EnhancedscDeepCluster(
  (dropoutLayer): Dropout(p=0.8, inplace=False)
  (encoder): Sequential(
    (0): Linear(in_features=16653, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
  )
  (attention): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Sequential(
    (0): Linear(in_featur

Pretraining stage.

In [8]:
t0 = time()
if args.ae_weights is None:
    model.pretrain_autoencoder(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, 
                            batch_size=args.batch_size, epochs=args.pretrain_epochs, ae_weights=args.ae_weight_file)
else:
    if os.path.isfile(args.ae_weights):
        print("==> loading checkpoint '{}'".format(args.ae_weights))
        checkpoint = torch.load(args.ae_weights)
        model.load_state_dict(checkpoint['ae_state_dict'])
    else:
        print("==> no checkpoint found at '{}'".format(args.ae_weights))
        raise ValueError

print('Pretraining time: %d seconds.' % int(time() - t0))

==> loading checkpoint '10xPBMC_with_attn_weights.pth.tar'
Pretraining time: 0 seconds.


Pretrain epoch 196, ZINB loss: 0.22281503
Pretrain epoch 197, ZINB loss: 0.22263191
Pretrain epoch 198, ZINB loss: 0.22256174
Pretrain epoch 199, ZINB loss: 0.22243141
Pretrain epoch 200, ZINB loss: 0.22252310
Pretrain epoch 201, ZINB loss: 0.22236248
Pretrain epoch 202, ZINB loss: 0.22244692
Pretrain epoch 203, ZINB loss: 0.22217898
Pretrain epoch 204, ZINB loss: 0.22209356
Pretrain epoch 205, ZINB loss: 0.22210407
Pretrain epoch 206, ZINB loss: 0.22195778
Pretrain epoch 207, ZINB loss: 0.22193105
Pretrain epoch 208, ZINB loss: 0.22195967
Pretrain epoch 209, ZINB loss: 0.22182915
Pretrain epoch 210, ZINB loss: 0.22169341
Pretrain epoch 211, ZINB loss: 0.22163370
Pretrain epoch 212, ZINB loss: 0.22177499
Pretrain epoch 213, ZINB loss: 0.22147643
Pretrain epoch 214, ZINB loss: 0.22148880
Pretrain epoch 215, ZINB loss: 0.22136244
Pretrain epoch 216, ZINB loss: 0.22121936
Pretrain epoch 217, ZINB loss: 0.22130410
Pretrain epoch 218, ZINB loss: 0.22126952
Pretrain epoch 219, ZINB loss: 0.2

Clustering stage.

In [9]:
if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

if args.n_clusters > 0:
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X.astype(np.float32), size_factor=adata.obs.size_factors, n_clusters=args.n_clusters, init_centroid=None, 
                y_pred_init=None, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)
else:
    cluster_centers, n_clusters, y_pred_init = model_preprocess.louvain_init_clustering(model=model, adata=adata, knn=args.knn, resolution=args.resolution)
    print('Estimated number of clusters: ', n_clusters)
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, n_clusters=n_clusters, init_centroid=cluster_centers, 
                y_pred_init=y_pred_init, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)


print('Total time: %d seconds.' % int(time() - t0))

Clustering stage


C:\Users\bhari\Downloads\EnhancedscDeepCluster\./EnhancedscDeepCluster\model.py:230: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  size_factor = torch.tensor(size_factor, dtype=torch.float64)


Initializing cluster centers with kmeans.
Initializing k-means: NMI= 0.7032, ARI= 0.6330
Clustering   1: NMI= 0.7032, ARI= 0.6330
Epoch   1: Total: 0.41969351 Clustering Loss: 0.18665691 ZINB Loss: 0.23303660
Clustering   2: NMI= 0.7058, ARI= 0.6246
Epoch   2: Total: 0.48050930 Clustering Loss: 0.24484997 ZINB Loss: 0.23565933
Clustering   3: NMI= 0.7212, ARI= 0.6369
Epoch   3: Total: 0.50090901 Clustering Loss: 0.26486091 ZINB Loss: 0.23604809
Clustering   4: NMI= 0.7177, ARI= 0.6189
Epoch   4: Total: 0.51091165 Clustering Loss: 0.27372247 ZINB Loss: 0.23718918
Clustering   5: NMI= 0.7282, ARI= 0.6232
Epoch   5: Total: 0.48626468 Clustering Loss: 0.24960260 ZINB Loss: 0.23666209
Clustering   6: NMI= 0.7184, ARI= 0.6155
Epoch   6: Total: 0.51626250 Clustering Loss: 0.27875445 ZINB Loss: 0.23750804
Clustering   7: NMI= 0.7288, ARI= 0.6230
Epoch   7: Total: 0.46770886 Clustering Loss: 0.23032190 ZINB Loss: 0.23738696
Clustering   8: NMI= 0.7235, ARI= 0.6191
Epoch   8: Total: 0.52909985 C

In [9]:
if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

args.n_clusters = 0

if args.n_clusters > 0:
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X.astype(np.float32), size_factor=adata.obs.size_factors, n_clusters=args.n_clusters, init_centroid=None, 
                y_pred_init=None, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)
else:
    cluster_centers, n_clusters, y_pred_init = model_preprocess.louvain_init_clustering(model=model, adata=adata, knn=args.knn, resolution=args.resolution)
    print('Estimated number of clusters: ', n_clusters)
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, n_clusters=n_clusters, init_centroid=cluster_centers, 
                y_pred_init=y_pred_init, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)


print('Total time: %d seconds.' % int(time() - t0))

C:\Users\bhari\Downloads\EnhancedscDeepCluster\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Estimated number of clusters:  13
Clustering stage


C:\Users\bhari\Downloads\EnhancedscDeepCluster\./EnhancedscDeepCluster\model.py:230: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  size_factor = torch.tensor(size_factor, dtype=torch.float64)


Initializing cluster centers with kmeans.
Initializing k-means: NMI= 0.6908, ARI= 0.4810
Clustering   1: NMI= 0.6925, ARI= 0.4766
Epoch   1: Total: 0.50454393 Clustering Loss: 0.27135825 ZINB Loss: 0.23318568
Clustering   2: NMI= 0.6869, ARI= 0.4965
Epoch   2: Total: 0.54942749 Clustering Loss: 0.31568833 ZINB Loss: 0.23373917
Clustering   3: NMI= 0.6932, ARI= 0.4811
Epoch   3: Total: 0.53770469 Clustering Loss: 0.30369097 ZINB Loss: 0.23401372
Clustering   4: NMI= 0.6896, ARI= 0.4803
Epoch   4: Total: 0.54252399 Clustering Loss: 0.30864393 ZINB Loss: 0.23388006
Clustering   5: NMI= 0.6969, ARI= 0.4835
Epoch   5: Total: 0.57899771 Clustering Loss: 0.34368819 ZINB Loss: 0.23530952
Clustering   6: NMI= 0.6921, ARI= 0.4786
Epoch   6: Total: 0.59556295 Clustering Loss: 0.36054481 ZINB Loss: 0.23501814
Clustering   7: NMI= 0.6964, ARI= 0.4789
Epoch   7: Total: 0.61556164 Clustering Loss: 0.37970253 ZINB Loss: 0.23585911
Clustering   8: NMI= 0.6915, ARI= 0.4768
Epoch   8: Total: 0.63832228 C

Output and save predicted labels and latent features.

In [11]:
if y is not None:
    nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    ss = np.round(metrics.silhouette_score(adata.X, y_pred), 5)
    ch = np.round(metrics.calinski_harabasz_score(adata.X, y_pred), 2)
    print('Evaluating cells:')
    print(f'  NMI = {nmi:.4f}')
    print(f'  ARI = {ari:.4f}')
    print(f'  Silhouette Score = {ss:.4f}')
    print(f'  Calinski-Harabasz Index = {ch:.2f}')

final_latent = model.encodeBatch(torch.tensor(adata.X)).cpu().numpy()
np.savetxt(args.final_latent_file, final_latent, delimiter=",")
np.savetxt(args.predict_label_file, y_pred, delimiter=",", fmt="%i")

Evaluating cells:
  NMI = 0.7279
  ARI = 0.6173
  Silhouette Score = -0.0244
  Calinski-Harabasz Index = 14.28


Run t-SNE on latent features.

In [ ]:
from openTSNE import TSNE

tsne_embedding = TSNE(
                    perplexity=30,
                    initialization="pca",
                    metric="euclidean",
                    n_jobs=8,
                    random_state=42,
                )
latent_tsne_2 = tsne_embedding.fit(final_latent)
np.savetxt("tsne_2D.txt", latent_tsne_2, delimiter=",")

Plot 2D t-SNE of latent features

In [ ]:
rm(list = ls())
library(ggplot2)

latent_tsne <- read.table("tsne_2D.txt", sep=",")
colnames(latent_tsne) <- c("TSNE_1", "TSNE_2")
y_pred <- as.numeric(readLines("pred_labels.txt"))
y_pred <- factor(y_pred, levels=0:max(y_pred))

dat <- data.frame(latent_tsne, y_pred=y_pred)

ggplot(dat, aes(x=TSNE_1, y=TSNE_2, color=y_pred)) + geom_point() + theme_classic()